In [2]:
import pandas as pd
import numpy as np
import json

In [5]:
df = pd.read_json('/Users/allen/Downloads/IMDB_movie_details.json', lines=True)

In [6]:
df

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...",1h 57min,"[Action, Thriller]",6.9,1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in..."
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",1h 45min,[Comedy],6.6,2013-11-01,Four boys around the age of 10 are friends in ...
2,tt0243655,"The setting is Camp Firewood, the year 1981. I...",1h 37min,"[Comedy, Romance]",6.7,2002-04-11,
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...",2h 6min,"[Adventure, Drama, Western]",8.3,1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...
4,tt0126886,Tracy Flick is running unopposed for this year...,1h 43min,"[Comedy, Drama, Romance]",7.3,1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...
...,...,...,...,...,...,...,...
1567,tt0289879,Evan Treborn grows up in a small town with his...,1h 53min,"[Sci-Fi, Thriller]",7.7,2004-01-23,"In the year 1998, Evan Treborn (Ashton Kutcher..."
1568,tt1723811,Brandon is a 30-something man living in New Yo...,1h 41min,[Drama],7.2,2012-01-13,"Brandon (Michael Fassbender) is a successful, ..."
1569,tt5013056,Evacuation of Allied soldiers from the British...,1h 46min,"[Action, Drama, History]",8.1,2017-07-21,The film alternates between three different pe...
1570,tt0104014/,"For a while now, beautiful 24-year-old Diana B...",1h 33min,"[Comedy, Drama]",5.3,1992-02-21,


In [8]:
reviews = pd.read_json('/Users/allen/Downloads/IMDB_reviews.json', lines=True)

In [9]:
reviews.head()

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"


In [12]:
reviews[reviews['movie_id'] == 'tt0111161']

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"
...,...,...,...,...,...,...,...
4356,4 October 2000,tt0111161,ur0690751,False,Some movies really need more than 2 hours to d...,7,"Unnecessarily long, but still excellent."
4357,4 August 2000,tt0111161,ur0819902,False,With all due respect to the people that compla...,10,Quite possibly the best movie ever made
4358,26 July 2000,tt0111161,ur0562732,False,Every single scene in The Shawshank Redemption...,9,Every once in a long while a truly inspiring f...
4359,26 February 2004,tt0111161,ur2693433,False,"At the beginning i thought, this would be an a...",10,The second place (Top 250) is unfair for this ...


In [13]:
df[df['movie_id'] == 'tt0111161']

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis
1263,tt0111161,Chronicles the experiences of a formerly succe...,2h 22min,"[Crime, Drama]",9.3,1994-10-14,"In 1947, Andy Dufresne (Tim Robbins), a banker..."


In [14]:
a = pd.read_csv('/Users/allen/Downloads/title.akas.tsv', sep='\t', nrows = 10)

In [15]:
a

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
1,tt0000001,2,Καρμενσίτα,GR,\N,\N,\N,0
2,tt0000001,3,Карменсита,RU,\N,\N,\N,0
3,tt0000001,4,Carmencita,US,\N,\N,\N,0
4,tt0000001,5,Carmencita,\N,\N,original,\N,1
5,tt0000002,1,Le clown et ses chiens,\N,\N,original,\N,1
6,tt0000002,2,A bohóc és kutyái,HU,\N,imdbDisplay,\N,0
7,tt0000002,3,Le clown et ses chiens,FR,\N,\N,\N,0
8,tt0000002,4,Clovnul si cainii sai,RO,\N,imdbDisplay,\N,0
9,tt0000002,5,Клоун и его собаки,RU,\N,\N,\N,0


In [18]:
b = pd.read_csv('/Users/allen/Downloads/title.principals.tsv.gz', sep='\t', nrows = 1000)

In [19]:
b

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Herself""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
...,...,...,...,...,...,...
995,tt0000556,2,nm0488932,actor,\N,"[""Boyfriend""]"
996,tt0000556,3,nm0676941,actor,\N,"[""Seducer""]"
997,tt0000556,4,nm0647763,cinematographer,\N,\N
998,tt0000557,10,nm0166633,actor,\N,\N
